# Представление результатов распознавания

In [1]:
from pathlib import Path
import pandas as pd
import pysubs2
import IPython

## Определение параметров

In [2]:
SCP_PATH = 'data/example_segments.scp'
U2S_PATH = 'data/utt2spk'
SEGMS_PATH = 'data/segments'
TRANS_PATH = 'data/transcriptions'
ASS_PATH = 'data/example.ass'
CSV_PATH = 'data/example.csv'

## Формирование субтитров

In [3]:
def make_ass(wav, segments, transcriptions, utt2spk, ass):
    """
    Формирование .ASS файла из транскрибаций
    
    Аргументы:
       wav: наименование аудио файла
       segments: путь к файлу описания сегментов
       transcriptions: путь к файлу транскрибации
       utt2spk: путь к файлу сопоставления сегментов и говорящих
       ass: путь к .ASS файлу субтитров
    """
    sub = pysubs2.SSAFile()
    sub.info['Title'] = 'Default Aegisub file'
    sub.info['YCbCr Matrix'] = 'None'
    sub.aegisub_project['Audio File'] = wav
    sub.aegisub_project['Scroll Position'] = 0
    sub.aegisub_project['Active Line'] = 0
    segments_df = pd.read_csv(segments, header=None, sep=' ', names=['utt_id', 'wav', 'start', 'end'])
    transcriptions_df = pd.read_csv(transcriptions, sep='\t', header=None, names=['utt_id', 'text'])
    utt2spk_df = pd.read_csv(utt2spk, sep='\t', header=None, names=['utt_id', 'speaker'])
    events = segments_df.merge(transcriptions_df, how='left', on='utt_id').merge(utt2spk_df, how='left', on='utt_id').fillna('')
    for row in events.values:
        event = pysubs2.SSAEvent(start=pysubs2.make_time(s=float(row[2])), end=pysubs2.make_time(s=float(row[3])), 
                                text=row[4], name=row[5])
        sub.events.append(event)
    sub.sort()
    sub.save(ass, format_='ass')
    return sub

In [4]:
sub = make_ass('example.wav', SEGMS_PATH, TRANS_PATH, U2S_PATH, ASS_PATH)
for event in sub.events:
    print(event)

<SSAEvent type=Dialogue start=0:00:00 end=0:00:07 text='наталья благодарю вас за ожиданьем по данному вопросу есть то во что она приведу на специалиста профиль на у подразделением'>
<SSAEvent type=Dialogue start=0:00:07 end=0:00:08 text='постарайтесь пожалуйста омовением'>
<SSAEvent type=Dialogue start=0:00:07 end=0:00:08 text='пять'>
<SSAEvent type=Dialogue start=0:00:09 end=0:00:12 text='я в свою очередь с вами прощайте всего дом проводов с питанием'>
<SSAEvent type=Dialogue start=0:00:12 end=0:00:13 text='бом'>


In [5]:
IPython.display.Audio('data/example.wav')

## Парсинг субтитров

In [5]:
def get_event_attributes(event):
    """
    Извлечение атрибутов события
    
    Аргументы:
        event: строка события
        
    Результат:
        attributes: словарь атрибутов события
    """
    attributes = {'Layer': event.layer,
                'Start': event.start,
                'End': event.end,
                'Style': event.style,
                'Name': event.name,
                'MarginL': event.marginl,
                'MarginR': event.marginr,
                'MarginV': event.marginv,
                'Effect': event.effect,
                'Text': event.text}
    return attributes

In [6]:
transcriptions = pd.DataFrame(columns=['Audio File', 'Start', 'End', 'Name', 'Text'])
for event in sub.events:
    attributes = get_event_attributes(event)
    attributes['Audio File'] = sub.aegisub_project['Audio File']
    transcriptions = transcriptions.append(pd.DataFrame(attributes, index=[0])[transcriptions.columns], ignore_index=True)
with open(CSV_PATH, 'w') as f:
    transcriptions.to_csv(f, header=False, index=False, encoding='cp1251')

In [7]:
transcriptions

,Audio File,Start,End,Name,Text
0,example.wav,780,7000,Канал 1,наталья благодарю вас за ожиданьем по данному ...
1,example.wav,7000,8860,Канал 1,постарайтесь пожалуйста омовением
2,example.wav,7460,8320,Канал 0,пять
3,example.wav,9400,12620,Канал 1,я в свою очередь с вами прощайте всего дом про...
4,example.wav,12770,13770,Канал 0,бом
